In [1]:
import os
import re
import glob
from typing import List

import numpy as np
import pandas as pd
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from kiwipiepy import Kiwi
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Config

In [2]:
base_dir = "/opt/data/"

kr_docs_dir = os.path.join(base_dir, "example_Kr_docs/")
movie_review_path = os.path.join(base_dir, "Korean_movie_reviews_2016.txt")

## Example1: Doc Similarity

### Read Data

In [3]:
paths = glob.glob(os.path.join(kr_docs_dir, "**", "**.txt"), recursive=True)
raw_docs = []
for p in paths:
    with open(p, "r", encoding="utf-8") as f:
        doc = f.read()
    raw_docs.append(doc)

print(f"num docs: {len(raw_docs)}")

num docs: 13


### Preprocess Data

In [4]:
def preprocess(
    content: str,
    tokenizer: Kiwi,
) -> List[str]:
    # replace characters which is not a subset of space or words or digits with " "
    replaced = re.sub(r"[^\s\w\d]", " ", content)
    tokens = tokenizer.tokenize(replaced)
    outputs = [token.form for token in tokens]
    return outputs

In [5]:
# tokenizer
kiwi = Kiwi()

In [6]:
# preprocess
docs = [preprocess(doc, tokenizer=kiwi) for doc in raw_docs]

print("==> Example: ")
print(docs[0])

==> Example: 
['정기', '석', '코로나', '19', '특별', '대응', '단장', '겸', '국가', '감염병', '위기', '대응', '자문', '위원장', '은', '21', '일', '접종', '부작용', '을', '어느', '정도', '는', '감수', '하', '고', '맞', '을', '가치', '가', '있', '다', '이', '라면서', '겨울철', '코로나', '19', '백신', '접종', '을', '독려', '하', '었', '다', '정', '위원장', '은', '이날', '코로나', '19', '정례', '브리핑', '에서', '기존', '에', '겨울', '에', '가장', '위험', '하', '었', '던', '독감', '보다', '코로나', '19', '가', '더', '위험', '하', 'ᆫ', '감염병', '으로', '이제', '코로나', '19', '백신', '은', '몇', '차', '접종', '개념', '이', '아니', '라', '겨울', '이', '되', '면', '되', '면', '맞', '는', '위험', '회피', '수단', '이', '라며', '이', '같이', '강조', '하', '었', '다', '정', '위원장', '은', '코로나', '19', '누적', '사망자', '가', '3', '만', '명', '을', '넘기', 'ᆫ', '것', '에', '대하', '어', '연', '평균', '코로나', '19', '사망자', '가', '독감', '으로', '인하', 'ᆫ', '사망자', '의', '100', '배', '를', '넘', '는', '셈', '이', 'ᆫ데', '아직', '도', '코로나', '19', '백신', '접종', '률', '이', '독감', '백신', '접종', '률', '에', '크', '게', '못', '미치', '어', '아쉽', '다', '고', '지적', '하', '었', '다', '정', '위원장', '은', '코로나', '19', '백신', '이', '감

In [7]:
tagged_docs = [TaggedDocument(doc, tags=[i]) for i, doc in enumerate(docs)]

print("==> Example: ")
print(tagged_docs[0])

==> Example: 
TaggedDocument<['정기', '석', '코로나', '19', '특별', '대응', '단장', '겸', '국가', '감염병', '위기', '대응', '자문', '위원장', '은', '21', '일', '접종', '부작용', '을', '어느', '정도', '는', '감수', '하', '고', '맞', '을', '가치', '가', '있', '다', '이', '라면서', '겨울철', '코로나', '19', '백신', '접종', '을', '독려', '하', '었', '다', '정', '위원장', '은', '이날', '코로나', '19', '정례', '브리핑', '에서', '기존', '에', '겨울', '에', '가장', '위험', '하', '었', '던', '독감', '보다', '코로나', '19', '가', '더', '위험', '하', 'ᆫ', '감염병', '으로', '이제', '코로나', '19', '백신', '은', '몇', '차', '접종', '개념', '이', '아니', '라', '겨울', '이', '되', '면', '되', '면', '맞', '는', '위험', '회피', '수단', '이', '라며', '이', '같이', '강조', '하', '었', '다', '정', '위원장', '은', '코로나', '19', '누적', '사망자', '가', '3', '만', '명', '을', '넘기', 'ᆫ', '것', '에', '대하', '어', '연', '평균', '코로나', '19', '사망자', '가', '독감', '으로', '인하', 'ᆫ', '사망자', '의', '100', '배', '를', '넘', '는', '셈', '이', 'ᆫ데', '아직', '도', '코로나', '19', '백신', '접종', '률', '이', '독감', '백신', '접종', '률', '에', '크', '게', '못', '미치', '어', '아쉽', '다', '고', '지적', '하', '었', '다', '정', '위원장', '은', '코로나', '19'

### Train Model
- `vector_size`: embedding dimension
- `dm`:
    - 1: applies Distributed Memory
        - similar to CBOW in Word2Vec, DM use neighboring words to predict the target word
    - 0: applies Distributed Bag of Words(DBOW)
        - similar to skip-gram in Word2Vec, DBOW uses document specific inputs and predicts whether an arbitrary word is in the document or not
- `min_count`: threshold to exclude words where a count is lower than min_count
- `negative`: negative sampling word countt

In [8]:
model = Doc2Vec(
    documents=tagged_docs,
    vector_size=100,
    dm=1,
    min_count=3,
    negative=5,
    alpha=0.001,
    epochs=1000,
)

In [9]:
# 13 = # of documents
len(model.dv)

13

In [10]:
model.dv[0].shape

(100,)

In [11]:
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    num = v1.dot(v2)
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)
    return num / (denom+1e-6)

In [12]:
cosine_similarity(model.dv[0], model.dv[1])

0.19172255952455314

In [13]:
# print examples

doc_id = 0

pos_examples = model.dv.most_similar(positive=doc_id, topn=2)
neg_examples = model.dv.most_similar(negative=doc_id, topn=2)

print(f"Input: {raw_docs[doc_id]}")

print("\n\n==> Positive Examples")
for pos_doc_id, _ in pos_examples:
    print(raw_docs[pos_doc_id])
    print("\n")

print("\n\n==> Negative Examples")
for neg_doc_id, _ in neg_examples:
    print(raw_docs[neg_doc_id])
    print("\n")

Input: 정기석 코로나19 특별대응단장 겸 국가감염병위기대응자문위원장은 21일 "(접종) 부작용을 어느 정도는 감수하고 맞을 가치가 있다"라면서 겨울철 코로나19 백신 접종을 독려했다.

정 위원장은 이날 코로나19 정례브리핑에서 "기존에 겨울에 가장 위험했던 독감보다 코로나19가 더 위험한 감염병으로, 이제 코로나19 백신은 '몇차 접종' 개념이 아니라 겨울이 되면 되면 맞는 위험 회피 수단"이라며 이같이 강조했다.

정 위원장은 코로나19 누적 사망자가 3만명을 넘긴 것에 대해 "연평균 코로나19 사망자가 독감으로 인한 사망자의 100배를 넘는 셈인데 아직도 코로나19 백신 접종률이 독감 백신 접종률에 크게 못미쳐 아쉽다"고 지적했다.

정 위원장은 코로나19 백신이 감염과 중증화 및 사망 뿐만 아니라 감염으로 인한 급성심근경색, 뇌졸중 등 후유증도 감소시킨다는 연구 결과를 인용하며 "특히 개량백신이 후유증을 줄일 수 있다는 연구 결과도 있고 우수성이 예측되는 만큼 아직 고위험군은 반드시 맞아주길 바란다"고 말했다.

또 재감염시 사망 위험이 2배, 입원 확률이 3배라는 미국 보건부의 연구 결과 등을 제시하면서 "코로나19는 감염 횟수가 많아질 수록 위험도가 훨씬 올라가는 만큼, 재감염 예방을 위해서도 백신 접종이 필요하다"고 밝혔다.

백신 부작용 우려에 대해선 "전 세계적으로 130억회분이 접종됐으나 안전성 문제로 백신 접종 정책을 달리한 나라는 없다"며 "새로운 백신이나 약제에 대한 두려움은 누구에게나 다 있지만 이렇게 수 많은 백신이 큰 문제없이 접종되며 전 세계적으로 인정됐고 부작용 이슈는 이제는 많이 안정됐다"고 강조했다.

이어 "백신을 비롯한 모든 약제 등 몸에 들어가는 이물질은 절대 안전하지 않지만 어느 정도 그 위험을 무릅쓰고라도 위험과 예방접종이나 약, 시술·수술 등의 형평성을 따져서 의료행위를 한다"며 "개량백신은 어느 정도 위험을 감수하고 맞을 가치가 있다"고 말했다.

정 위원장은 요양병원 등 감염취약시설의 코로나19 추가 접종

## Example2: Sentiment Analysis

### Read Data

In [14]:
docs = []
labels = []
exceptions = []
with open(movie_review_path, "r") as f:
    for line in f:
        content = line.strip()
        try:
            doc, label = content.split("\t")
        except Exception as e:
            exceptions.append([content, str(e)])
        docs.append(doc)
        labels.append(int(label))

In [15]:
print(f"num docs: {len(docs)}")
print(f"num exceptions: {len(exceptions)}")

num docs: 166039
num exceptions: 655


In [16]:
exceptions[:5]

[['1', 'not enough values to unpack (expected 2, got 1)'],
 ['1', 'not enough values to unpack (expected 2, got 1)'],
 ['0', 'not enough values to unpack (expected 2, got 1)'],
 ['1', 'not enough values to unpack (expected 2, got 1)'],
 ['0', 'not enough values to unpack (expected 2, got 1)']]

### Preprocess Data

In [17]:
doc_words = [doc.strip().split() for doc in docs]

In [18]:
docs[:2]

['부산 행 때문 너무 기대하고 봤', '한국 좀비 영화 어색하지 않게 만들어졌 놀랍']

In [19]:
doc_words[:2]

[['부산', '행', '때문', '너무', '기대하고', '봤'],
 ['한국', '좀비', '영화', '어색하지', '않게', '만들어졌', '놀랍']]

In [20]:
tagged_docs = [TaggedDocument(doc, tags=[i]) for i, doc in enumerate(doc_words)]

In [21]:
tagged_docs[:2]

[TaggedDocument(words=['부산', '행', '때문', '너무', '기대하고', '봤'], tags=[0]),
 TaggedDocument(words=['한국', '좀비', '영화', '어색하지', '않게', '만들어졌', '놀랍'], tags=[1])]

### Train Model

In [22]:
model = Doc2Vec(
    documents=tagged_docs,
    vector_size=100,
    min_count=3,
    window=3,
    epochs=30,
    dm=1,
    negative=5,
    alpha=0.001,
    workers=os.cpu_count(),
)

In [23]:
raw_docs = docs

In [24]:
# print examples

doc_id = 0

pos_examples = model.dv.most_similar(positive=doc_id, topn=2)
neg_examples = model.dv.most_similar(negative=doc_id, topn=2)

print(f"Input: {raw_docs[doc_id]}")

print("\n\n==> Positive Examples")
for pos_doc_id, _ in pos_examples:
    print(raw_docs[pos_doc_id])
    print("\n")

print("\n\n==> Negative Examples")
for neg_doc_id, _ in neg_examples:
    print(raw_docs[neg_doc_id])
    print("\n")

Input: 부산 행 때문 너무 기대하고 봤


==> Positive Examples
노잼 노잼 노잼 개핵 노잼 보지 마시


인생 영화 가장 짙은 향수




==> Negative Examples
예고편 제일 재밌다 스토리 전개 모두 한숨 나옴 매력 있는 캐릭터 연기 잘하는 배우 좋 설정 가지 엉망 만들어


지루한 하나 정말 봤 여운 남는 영화 입니 잔인 영화 만들 이런 영화 봅




### Train Classification Model

In [25]:
# 100 = embedding dim

doc_vectors = model.dv.vectors

print(doc_vectors.shape)

(166039, 100)


In [26]:
# train/test split

X_train, X_test, y_train, y_test = train_test_split(doc_vectors, labels, test_size=0.2, random_state=123)

In [27]:
# train classifier

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [28]:
y_pred = clf.predict(X_test)

In [29]:
accuracy_score(y_test, y_pred)

0.7114851842929415